In [2]:
import pandas as pd
#from mlxtend.preprocessing import OnehotTransactions
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [5]:
df = pd.read_csv('../data/dataset-import-normal-sin.csv', sep='|')
df.head()
df['gestionstr'] = df['gestion'].astype(str)
df['messtr'] = df['mes'].astype(str)
df['item'] = df['gestionstr'].str.cat(df['messtr'], sep="_")
#filtrado de un año
df1 = df[df['gestion'] == 2019]
print(df1.head())
print(df1.shape)

      gestion  mes    producto gestionstr messtr    item
3671     2019    1      Huevos       2019      1  2019_1
3672     2019    1     Filetes       2019      1  2019_1
3673     2019    1  Lactosuero       2019      1  2019_1
3674     2019    1       Carne       2019      1  2019_1
3675     2019    1  Crustáceos       2019      1  2019_1
(144, 6)


In [6]:
df2 = df1.groupby('item')['producto'].apply(list)
df2

item
2019_1     [Huevos, Filetes, Lactosuero, Carne, Crustáceo...
2019_10    [Pescado, Huevos, Leche, Filetes, Suero, Lacto...
2019_11    [Lactosuero, Pescado, Moluscos, Leche, Carne, ...
2019_12    [Filetes, Crustáceos, Leche, Quesos, Pescado, ...
2019_2     [Crustáceos, Lactosuero, Mantequilla, Huevos, ...
2019_3     [Leche, Filetes, Lactosuero, Mantequilla, Trip...
2019_4     [Tripas, Crustáceos, Carne, Filetes, Moluscos,...
2019_5     [Moluscos, Mantequilla, Huevos, Suero, Pescado...
2019_6     [Lactosuero, Pescado, Leche, Moluscos, Carne, ...
2019_7     [Huevos, Lactosuero, Suero, Moluscos, Tripas, ...
2019_8     [Quesos, Lactosuero, Suero, Huevos, Mantequill...
2019_9     [Mantequilla, Tocino, Carne, Quesos, Pescado, ...
Name: producto, dtype: object

In [7]:
oht = TransactionEncoder()
oht_ary = oht.fit(df2).transform(df2)
data  = pd.DataFrame(oht_ary, columns = oht.columns_)
print(data)
print(data.shape)

    Ambar  Carne  Crustáceos  Filetes  Huevos  Lactosuero  Leche  Mantequilla  \
0   False   True        True     True    True        True   True         True   
1   False   True        True     True    True        True   True         True   
2   False   True        True     True    True        True   True        False   
3   False   True        True     True    True        True   True         True   
4    True   True        True     True    True        True   True         True   
5   False   True        True     True    True        True   True         True   
6   False   True        True     True   False        True   True         True   
7   False   True        True    False    True        True   True         True   
8   False   True        True     True    True        True   True         True   
9   False   True        True     True    True        True   True         True   
10  False   True        True     True    True        True   True         True   
11  False   True        True

In [8]:
frequent_itemsets = apriori(data, min_support=0.6, use_colnames=True)
print(frequent_itemsets)

       support                                           itemsets
0     1.000000                                            (Carne)
1     1.000000                                       (Crustáceos)
2     0.916667                                          (Filetes)
3     0.916667                                           (Huevos)
4     1.000000                                       (Lactosuero)
...        ...                                                ...
2042  0.833333  (Moluscos, Pescado, Filetes, Suero, Crustáceos...
2043  0.833333  (Moluscos, Pescado, Quesos, Suero, Crustáceos,...
2044  0.750000  (Moluscos, Pescado, Filetes, Quesos, Suero, Ma...
2045  0.750000  (Moluscos, Pescado, Filetes, Quesos, Suero, Cr...
2046  0.750000  (Moluscos, Pescado, Filetes, Quesos, Suero, Cr...

[2047 rows x 2 columns]


In [9]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
print(rules)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []


In [11]:
frequent_itemsets = apriori(data, min_support=0.0053, use_colnames=True) # min_support = 0.07
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Carne),(Ambar),1.000000,0.083333,0.083333,0.083333,1.000000,0.000000,1.000000
1,(Ambar),(Carne),0.083333,1.000000,0.083333,1.000000,1.000000,0.000000,inf
2,(Ambar),(Crustáceos),0.083333,1.000000,0.083333,1.000000,1.000000,0.000000,inf
3,(Crustáceos),(Ambar),1.000000,0.083333,0.083333,0.083333,1.000000,0.000000,1.000000
4,(Ambar),(Filetes),0.083333,0.916667,0.083333,1.000000,1.090909,0.006944,inf
5,(Filetes),(Ambar),0.916667,0.083333,0.083333,0.090909,1.090909,0.006944,1.008333
6,(Ambar),(Huevos),0.083333,0.916667,0.083333,1.000000,1.090909,0.006944,inf
7,(Huevos),(Ambar),0.916667,0.083333,0.083333,0.090909,1.090909,0.006944,1.008333
8,(Ambar),(Lactosuero),0.083333,1.000000,0.083333,1.000000,1.000000,0.000000,inf
9,(Lactosuero),(Ambar),1.000000,0.083333,0.083333,0.083333,1.000000,0.000000,1.000000


In [12]:
rules.tail(30) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3670072,"(Leche, Tocino)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.333333,0.083333,0.083333,0.250000,3.000000,0.055556,1.222222
3670073,"(Tocino, Miel)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.166667,0.250000,0.083333,0.500000,2.000000,0.041667,1.500000
3670074,"(Carne, Tocino)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.333333,0.083333,0.083333,0.250000,3.000000,0.055556,1.222222
3670075,"(Lactosuero, Tocino)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.333333,0.083333,0.083333,0.250000,3.000000,0.055556,1.222222
3670076,"(Huevos, Tocino)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.250000,0.166667,0.083333,0.333333,2.000000,0.041667,1.250000
3670077,"(Leche, Tripas)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.333333,0.083333,0.083333,0.250000,3.000000,0.055556,1.222222
3670078,"(Miel, Tripas)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.166667,0.250000,0.083333,0.500000,2.000000,0.041667,1.500000
3670079,"(Carne, Tripas)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.333333,0.083333,0.083333,0.250000,3.000000,0.055556,1.222222
3670080,"(Lactosuero, Tripas)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.333333,0.083333,0.083333,0.250000,3.000000,0.055556,1.222222
3670081,"(Huevos, Tripas)","(Moluscos, Pescado, Filetes, Quesos, Suero, Cr...",0.250000,0.166667,0.083333,0.333333,2.000000,0.041667,1.250000


In [14]:
print(rules.shape)

(3670102, 9)


In [15]:
rules[rules['confidence']>=.95]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Ambar),(Carne),0.083333,1.000000,0.083333,1.0,1.000000,0.000000,inf
2,(Ambar),(Crustáceos),0.083333,1.000000,0.083333,1.0,1.000000,0.000000,inf
4,(Ambar),(Filetes),0.083333,0.916667,0.083333,1.0,1.090909,0.006944,inf
6,(Ambar),(Huevos),0.083333,0.916667,0.083333,1.0,1.090909,0.006944,inf
8,(Ambar),(Lactosuero),0.083333,1.000000,0.083333,1.0,1.000000,0.000000,inf
...,...,...,...,...,...,...,...,...,...
3669683,"(Carne, Miel, Huevos, Tripas)","(Moluscos, Pescado, Filetes, Suero, Crustáceos...",0.083333,0.333333,0.083333,1.0,3.000000,0.055556,inf
3669684,"(Miel, Lactosuero, Huevos, Tripas)","(Moluscos, Pescado, Filetes, Suero, Crustáceos...",0.083333,0.333333,0.083333,1.0,3.000000,0.055556,inf
3669980,"(Huevos, Tocino, Tripas)","(Moluscos, Pescado, Filetes, Suero, Crustáceos...",0.083333,0.333333,0.083333,1.0,3.000000,0.055556,inf
3669987,"(Huevos, Tocino, Miel)","(Moluscos, Pescado, Filetes, Suero, Crustáceos...",0.083333,0.333333,0.083333,1.0,3.000000,0.055556,inf


# Conclusión

* Se puede apreciar que las reglas de asociación encontradas en la data de importaciones del año 2019, se encuentran que los productos envasados en almivar (duraznos en almivar, frutas en almivar, etc.)  estan asociados fuertemente con importación de productos cárnicos de origen animal y tambien se puede observar que impotar productos de origen animal implica importar productos de mar.